In [1]:
#using Pkg
#Pkg.add("Pandas")
using Pandas

In [2]:
dataset = read_csv("ecma_data.csv")

,Subject,G1P1,G1P2,G1P3,G1P4,G2P1,G2P2,G2P3,G2P4
0,1,1,3,2,3,1,1,2,3
1,2,2,3,2,2,2,3,3,3
2,3,2,1,1,1,1,2,2,2
3,4,3,1,2,2,1,2,2,3
4,5,3,2,2,1,3,1,1,3
5,6,1,2,2,1,3,2,2,3
6,7,1,2,1,1,1,2,3,3
7,8,1,2,1,1,1,2,3,3
8,9,1,2,1,1,1,2,3,3
9,10,1,1,1,1,1,2,1,3


In [3]:
length(query(dataset, :(G1P1==1&G1P2==2&G1P3==1&G1P4==1)))

62

In Game 1 every level of player other than level 0 chooses the profile of actions `1,2,1,1`.  Of the 80, 62 chose this profile.  This could randomly include some level 0 players.

Estimate the probability $\rho$ that a randomly selected player from this dataset will have level $0$,  To do this, let $\pi$ be the probability that a randomly selected player will choose the profile `1,2,1,1`.  Then $(1-\pi)$ is the probability he or she won't.  Since 
$$\pi = (1-\rho)+\rho (\frac{1}{3})^4$$ we could estimate $\pi$ then calculate an estimate of $\rho$ from that.

Given $\pi$, the probablity of seeing a particular collection of profiles, 62 of which are `1,2,1,1` while 18 aren't is
$$\pi^{62}(1-\pi)^{18}$$

In [4]:
using SymPy
pi,rho = symbols("pi, rho", positive=true)
prob = (pi^(62))*(1-pi)^(18)

 62        18
π  ⋅(1 - π)  

In [5]:
dprob = diff(prob,pi)

      62        17       61        18
- 18⋅π  ⋅(1 - π)   + 62⋅π  ⋅(1 - π)  

In [6]:
solve(dprob,pi)

2-element Vector{Sym}:
 31/40
     1

Using entry 2, the value for $\pi$ that maximizes the probability of seeing this data is $\frac{62}{80}$

Now to estimate $\rho$:

In [7]:
solve((62/80)-(1-rho)-rho*(1/3)^4,rho)

1-element Vector{Sym}:
 0.227812500000000

# Both Games

Assuming only the K-level explanation for behavior there are four profiles that we need to focus on:

Now we calculate profiles:
1. level 1: `1,2,1,1,1,2,1,3`
2. level 2: `1,2,1,1,1,2,2,3`
3. level 3: `1,2,1,1,1,1,2,3`
4. level 4:  `1,2,1,1,3,1,2,3`

Level 4 subject playing as Player 1 in game 2 best replies to the expectation that Player 3 is level 3 who plays 1. The best reply to that is action 3. A level 5 subject as player 1 in Game 2 has the same belief that Player 3 will play 1, so this pattern is repeating.  It is also the same pattern that a fully rational player is supposed to use.  So there is no information in the experiment that would make it possible to tell the difference between levels higher than 3, or whether those players were fully rational (or even Nash).

K level reasoning only explains $\frac{1}{3}$ of the data.

In [74]:
l1 = length(query(dataset, :(G1P1==1&G1P2==2&G1P3==1&G1P4==1&G2P1==1&G2P2==2&G2P3==1&G2P4==3)))
l2 = length(query(dataset, :(G1P1==1&G1P2==2&G1P3==1&G1P4==1&G2P1==1&G2P2==2&G2P3==2&G2P4==3)))
l3 = length(query(dataset, :(G1P1==1&G1P2==2&G1P3==1&G1P4==1&G2P1==1&G2P2==1&G2P3==2&G2P4==3)))
l4 = length(query(dataset, :(G1P1==1&G1P2==2&G1P3==1&G1P4==1&G2P1==3&G2P2==1&G2P3==2&G2P4==3)))
string(l1,"  ",l2,"  ",l3," ",l4)

"7  12  14 13"

In [75]:
l1+l2+l3+l4/80

33.1625

Yet even when a player plays one of the profiles listed above, we still aren't sure of their level because the level 0 player could accidentally choose that profile as well. 

So the best we can do is to find the probability with which each player has each different level.  So let $\rho_0$ be a probability for level 0 and $\rho_i$ be the probability for each of the other levels.  

Now suppose we choose a profile, say `1,2,1,1,1,1,2,3` which is the profile that a level 3 player is supposed to use.  The probability a randomly chosen subject will choose that profile is given by
$$
\sum_{i=0}^{4} \rho_i\xi_i 
$$
where $\xi_i$ is the probability that a player of level i plays that profile.

The only two types for which $\xi_i$ is positive are levels 3 and level 0.

On this basis we can start to do estimation.

In [80]:
rho_1,rho_2,rho_3,rho_4,pi_1,pi_2,pi_3,pi_4 = symbols("rho_1,rho_2,rho_3,rho_4,pi_1,pi_2,pi_3,pi_4", positive=true)

(rho_1, rho_2, rho_3, rho_4, pi_1, pi_2, pi_3, pi_4)

As above, let $\pi_i$ be the probability with which profile $i$ is played. Then if $\rho_i$ happened to be the actual probability with which a subject in the pool has level 1, then we could write the probability with which the level 1 action profile `1,2,1,1,1,2,3,3` is played as
$$
\pi_1 = \rho_1+\rho_0(\frac{1}{3})^8
$$
Similary for
$$
\pi_2 = \rho_2+\rho_0(\frac{1}{3})^8
$$
and
$$
\pi_3 = \rho_3+\rho_0(\frac{1}{3})^8
$$
and
$$
\pi_4 = \rho_4+\rho_0(\frac{1}{3})^8
$$

The probability of seeing the data is 
$$
\pi_1^7*\pi_2^{12}*\pi_3^{14}*\pi_4^{13}*(1-\pi_1-\pi_2-\pi_3-\pi_4)^{80-7-12-14}
$$

# First try: symbolic maximization

Choose values for $\pi_1$,$\pi_2$ and $\pi_3$ to maximize the probability of the data using the above formula.  This method is not a good one and probably won't work with symbolics.

In [81]:
prob_full = pi_1^7*pi_2^12*pi_3^14*pi_4^13*(1-pi_1-pi_2-pi_3-pi_4)^(80-46)

  7   12   14   13                         34
π₁ ⋅π₂  ⋅π₃  ⋅π₄  ⋅(-π₁ - π₂ - π₃ - π₄ + 1)  

In [83]:
dprob_full = [diff(prob_full,v) for v in [pi_1,pi_2,pi_3,pi_4]]

4-element Vector{Sym}:
  -34*pi_1^7*pi_2^12*pi_3^14*pi_4^13*(-pi_1 - pi_2 - pi_3 - pi_4 + 1)^33 + 7*pi_1^6*pi_2^12*pi_3^14*pi_4^13*(-pi_1 - pi_2 - pi_3 - pi_4 + 1)^34
 -34*pi_1^7*pi_2^12*pi_3^14*pi_4^13*(-pi_1 - pi_2 - pi_3 - pi_4 + 1)^33 + 12*pi_1^7*pi_2^11*pi_3^14*pi_4^13*(-pi_1 - pi_2 - pi_3 - pi_4 + 1)^34
 -34*pi_1^7*pi_2^12*pi_3^14*pi_4^13*(-pi_1 - pi_2 - pi_3 - pi_4 + 1)^33 + 14*pi_1^7*pi_2^12*pi_3^13*pi_4^13*(-pi_1 - pi_2 - pi_3 - pi_4 + 1)^34
 -34*pi_1^7*pi_2^12*pi_3^14*pi_4^13*(-pi_1 - pi_2 - pi_3 - pi_4 + 1)^33 + 13*pi_1^7*pi_2^12*pi_3^14*pi_4^12*(-pi_1 - pi_2 - pi_3 - pi_4 + 1)^34

This is the method that you can try, it will probably run on and on without finding a solution.  This is probably because it has too many solutions.

In [84]:
#solve(dprob_full,pi_1,pi_2,pi_3,pi_4)

# Method 2 - Numerical

Write down a traditional likelihood function.  It is just the natural log of the `prob_full` symbolic function converted into a function that contains no symbolic stuff.  

In [85]:
llikelihood(x) = (-1)*(7*log(x[1])+12*log(x[2])+14*log(x[3])+13*log(x[4])+(80-7-12-14-13)*log(1-x[1]-x[2]-x[3]-x[4]))


llikelihood (generic function with 1 method)

In [86]:
llikelihood([.3,.3,.1,.2,])

154.32226060755028

In [69]:
import Pkg
for package in ["BlackBoxOptim", "Distributions", "ForwardDiff", "JSON", "Optim", "Quadrature", "StatsPlots",
    "DotEnv","MySQL","DBInterface","Tables","HTTP", "JSON"]
    if false
        Pkg.add(package)
    end
end
using Optim

In [87]:
l = [0.05,0.05,0.05,0.05]
u = [0.95,0.95,0.95,0.95]
x0 = [.3,.3,.1,.2]
result = optimize(llikelihood,l,u,x0,Fminbox())

 * Status: success

 * Candidate solution
    Final objective value:     1.169345e+02

 * Found with
    Algorithm:     Fminbox with L-BFGS

 * Convergence measures
    |x - x'|               = 6.07e-10 ≰ 0.0e+00
    |x - x'|/|x'|          = 2.06e-09 ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 7.04e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    4
    f(x) calls:    59
    ∇f(x) calls:   59


In [88]:
parameters = Optim.minimizer(result)

4-element Vector{Float64}:
 0.08750000010222761
 0.1500000000348374
 0.17499999998875343
 0.16250000001220438

Now we have the estimates for the three $\pi$, `.0875`, `.15` and `.175` and `.1625` (all give of take a bit).  As you should probably realize, `.0875` is just $7/80$, so this just illustrates the method.  Now we can find the estimates for the $\rho$.

In [89]:
solve([
        rho_1+(1-rho_1-rho_2-rho_3-rho_4)*(1/3)^8-parameters[1],
        rho_2+(1-rho_1-rho_2-rho_3-rho_4)*(1/3)^8-parameters[2],
        rho_3+(1-rho_1-rho_2-rho_3-rho_4)*(1/3)^8-parameters[3],
        rho_4+(1-rho_1-rho_2-rho_3-rho_4)*(1/3)^8-parameters[4],
        ], rho_1,rho_2,rho_3,rho_4
    )

Dict{Any, Any} with 4 entries:
  rho_2 => 0.149935183807926
  rho_4 => 0.162435183785293
  rho_1 => 0.0874351838753156
  rho_3 => 0.174935183761842

Note that each of the solutions is slightly smaller than the probability with which the corresponding profile appears.  This is intuitively consistent with the idea that playing the profile is incomplete proof that the subject has the corresponding type. 

Finally, we can find the value of the likelihood when we set the $\pi_i$ at their estimated values.

In [91]:
Optim.minimum(result)

116.93447783450424

What does that mean?  You have really just estimated the parameters of a multinomial distribution (the $\pi_i$) using 80 data points.  You can look up the varaince covariance matrix for these estimates on wikipedia. (Or maybe you remember it from your econometrics class).